# 2.5d Cutting model baseline [training]
Forked from: https://www.kaggle.com/code/yoyobar/2-5d-cutting-model-baseline-training

**This code is base on [2.5d segmentaion baseline [training]](https://www.kaggle.com/code/tanakar/2-5d-segmentaion-baseline-training)**

1. *     using kidney_1_dense for training and kidney_3_dense for val
2. *     image_size = 512
3. *     using DiceLoss
4. *     norm_with_clip
5. *     fix some bug


* This version is correspond with [2.5d Cutting model baseline [inference]](https://www.kaggle.com/code/yoyobar/2-5d-cutting-model-baseline-inference) version3



According to my experiments, using kidney_1_dense for training and kidney_3_dense for val is the best. You can even get 0.757, but using 2d model(se_resnext50_32x4d), you can set CFG.in_chans=1 to make this notebook as a 2d model training notebook.'

# Import

In [1]:
!mkdir -p /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/se-net-pretrained-imagenet-weights/* /root/.cache/torch/hub/checkpoints/
import torch as tc 
import torch.nn as nn  
import numpy as np
from tqdm import tqdm
import os,sys,cv2
from torch.cuda.amp import autocast
import matplotlib.pyplot as plt
!pip install -q albumentations
import albumentations as A
!python -m pip install --no-index --find-links=/kaggle/input/pip-download-for-segmentation-models-pytorch segmentation-models-pytorch
import segmentation_models_pytorch as smp
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from torch.nn.parallel import DataParallel
from glob import glob

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Looking in links: /kaggle/input/pip-download-for-segmentation-models-pytorch
Processing /kaggle/input/pip-download-for-segmentation-models-pytorch/segmentation_models_pytorch-0.3.3-py3-none-any.whl
Processing /kaggle/input/pip-download-for-segmentation-models-pytorch/pretrainedmodels-0.7.4.tar.gz (from segmentation-models-pytorch)
  Preparing metadata (setup.py) ... - \ done
Processing /kaggle/input/pip-download-for-segmentation-models-pytorch/efficientnet_pytorch-0.7.1.tar.gz (from segmentation-models-pytorch)
  Preparing metadata (setup.py) ... - done
Processing /kaggle/input/pip-download-for-segmentation-models-pytorch/timm-0.9.2-py3-none-any.whl (from segmentation-models-pytorch)
Processing /kaggle/input/pip-download-for-segmentation-models-pytorch/munch-4.0.0-py2.py3-none-any.whl (from pretrainedmodels==0.7.4->segmentation-models-pytorch)
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=a47f29dd1aa9711496a1c0fe8

# config

In [2]:
p_augm = 0.05 #0.5
#add rotate.  less p_augm

class CFG:
    # ============== pred target =============
    target_size = 1

    # ============== model CFG =============
    model_name = 'Unet'
    backbone = 'se_resnext50_32x4d'

    in_chans = 1   #5 # 65
    # ============== training CFG =============
    image_size = 512
    input_size = 512

    train_batch_size = 16
    valid_batch_size = train_batch_size * 2

    epochs = 25   #. 45 #30 #25
    lr = 6e-5
    chopping_percentile=1e-3
    # ============== fold =============
    valid_id = 1

    rotate_p = 0.02 #0.25 #0.5
    # ============== augmentation =============
    train_aug_list = [
        A.Rotate(limit=270, p= rotate_p),
        A.RandomScale(scale_limit=(0.8,1.25),interpolation=cv2.INTER_CUBIC,p=p_augm),
        A.RandomCrop(input_size, input_size,p=1),
        A.RandomGamma(p=p_augm*2/3),
        A.RandomBrightnessContrast(p=p_augm,),
        A.GaussianBlur(p=p_augm),
        A.MotionBlur(p=p_augm),
        A.GridDistortion(num_steps=5, distort_limit=0.3, p=p_augm),
        ToTensorV2(transpose_mask=True),
    ]
    train_aug = A.Compose(train_aug_list)
    valid_aug_list = [
        ToTensorV2(transpose_mask=True),
    ]
    valid_aug = A.Compose(valid_aug_list)

# Model

In [3]:
class CustomModel(nn.Module):
    def __init__(self, CFG, weight=None):
        super().__init__()
        self.model = smp.Unet(
            encoder_name=CFG.backbone, 
            encoder_weights=weight,
            in_channels=CFG.in_chans,
            classes=CFG.target_size,
            activation=None,
        )

    def forward(self, image):
        output = self.model(image)
        # output = output.squeeze(-1)
        return output[:,0]#.sigmoid()


def build_model(weight="imagenet"):
    from dotenv import load_dotenv
    load_dotenv()

    print('model_name', CFG.model_name)
    print('backbone', CFG.backbone)

    model = CustomModel(CFG, weight)

    return model.cuda()

# Functions

In [4]:
import torch as tc 

def min_max_normalization(x:tc.Tensor)->tc.Tensor:
    """input.shape=(batch,f1,...)"""
    shape=x.shape
    if x.ndim>2:
        x=x.reshape(x.shape[0],-1)
    
    min_=x.min(dim=-1,keepdim=True)[0]
    max_=x.max(dim=-1,keepdim=True)[0]
    if min_.mean()==0 and max_.mean()==1:
        return x.reshape(shape)
    
    x=(x-min_)/(max_-min_+1e-9)
    return x.reshape(shape)

def norm_with_clip(x:tc.Tensor,smooth=1e-5):
    dim=list(range(1,x.ndim))
    mean=x.mean(dim=dim,keepdim=True)
    std=x.std(dim=dim,keepdim=True)
    x=(x-mean)/(std+smooth)
    x[x>5]=(x[x>5]-5)*1e-3 +5
    x[x<-3]=(x[x<-3]+3)*1e-3-3
    return x

def add_noise(x:tc.Tensor,max_randn_rate=0.1,randn_rate=None,x_already_normed=False):
    """input.shape=(batch,f1,f2,...) output's var will be normalizate  """
    ndim=x.ndim-1
    if x_already_normed:
        x_std=tc.ones([x.shape[0]]+[1]*ndim,device=x.device,dtype=x.dtype)
        x_mean=tc.zeros([x.shape[0]]+[1]*ndim,device=x.device,dtype=x.dtype)
    else: 
        dim=list(range(1,x.ndim))
        x_std=x.std(dim=dim,keepdim=True)
        x_mean=x.mean(dim=dim,keepdim=True)
    if randn_rate is None:
        randn_rate=max_randn_rate*np.random.rand()*tc.rand(x_mean.shape,device=x.device,dtype=x.dtype)
    cache=(x_std**2+(x_std*randn_rate)**2)**0.5
    #https://blog.csdn.net/chaosir1991/article/details/106960408
    
    return (x-x_mean+tc.randn(size=x.shape,device=x.device,dtype=x.dtype)*randn_rate*x_std)/(cache+1e-7)

class Data_loader(Dataset):
    def __init__(self,paths,is_label, do_sort = True):
        
        self.paths=paths
        if do_sort==True:
           self.paths.sort()
        self.is_label=is_label
    
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self,index):
        img=cv2.imread(self.paths[index],cv2.IMREAD_GRAYSCALE)
        img=tc.from_numpy(img)
        if self.is_label:
            img=(img!=0).to(tc.uint8)*255
        else:
            img=img.to(tc.uint8)
        return img

def load_data(paths,is_label=False, do_sort = True):
    data_loader=Data_loader(paths,is_label, do_sort  ) 
    data_loader=DataLoader(data_loader, batch_size=16, num_workers=2)
    data=[]
    for x in tqdm(data_loader):
        data.append(x)
    x=tc.cat(data,dim=0)
    del data
    if not is_label:
        ########################################################################
        TH=x.reshape(-1).numpy()
        index = -int(len(TH) * CFG.chopping_percentile)
        TH:int = np.partition(TH, index)[index]
        x[x>TH]=int(TH)
        ########################################################################
        TH=x.reshape(-1).numpy()
        index = -int(len(TH) * CFG.chopping_percentile)
        TH:int = np.partition(TH, -index)[-index]
        x[x<TH]=int(TH)
        ########################################################################
        x=(min_max_normalization(x.to(tc.float16)[None])[0]*255).to(tc.uint8)
    return x


#https://www.kaggle.com/code/kashiwaba/sennet-hoa-train-unet-simple-baseline
def dice_coef(y_pred:tc.Tensor,y_true:tc.Tensor, thr=0.5, dim=(-1,-2), epsilon=0.001):
    y_pred=y_pred.sigmoid()
    y_true = y_true.to(tc.float32)
    y_pred = (y_pred>thr).to(tc.float32)
    inter = (y_true*y_pred).sum(dim=dim)
    den = y_true.sum(dim=dim) + y_pred.sum(dim=dim)
    dice = ((2*inter+epsilon)/(den+epsilon)).mean()
    return dice

class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = inputs.sigmoid()   
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        
        return 1 - dice

class Kaggld_Dataset(Dataset):
    def __init__(self,x:list,y:list,arg=False):
        super(Dataset,self).__init__()
        self.x=x#list[(C,H,W),...]
        self.y=y#list[(C,H,W),...]
        self.image_size=CFG.image_size
        self.in_chans=CFG.in_chans
        self.arg=arg
        if arg:
            self.transform=CFG.train_aug
        else: 
            self.transform=CFG.valid_aug

    def __len__(self) -> int:
        return sum([y.shape[0]-self.in_chans for y in self.y])
    
    def __getitem__(self,index):
        i=0
        for x in self.x:
            if index>x.shape[0]-self.in_chans:
                index-=x.shape[0]-self.in_chans
                i+=1
            else:
                break
        x=self.x[i]
        y=self.y[i]
        
        #print (f'x.shape[1] ={x.shape[1]}    x.shape[2]={x.shape[2]}')
        
        x_index= (x.shape[1]-self.image_size)//2 #np.random.randint(0,x.shape[1]-self.image_size)
        y_index= (x.shape[2]-self.image_size)//2 # np.random.randint(0,x.shape[2]-self.image_size)
        # i i+5 
        x=x[index:index+self.in_chans   ,   x_index:x_index+self.image_size,   y_index:y_index+self.image_size]
        # i+2
        y=y[index+self.in_chans//2   ,      x_index:x_index+self.image_size,   y_index:y_index+self.image_size]

        data = self.transform(image=x.numpy().transpose(1,2,0), mask=y.numpy())
        x = data['image']
        y = data['mask']>=127
        if self.arg:
            i=np.random.randint(4)
            x=x.rot90(i,dims=(1,2))
            y=y.rot90(i,dims=(0,1))
            for i in range(3):
                if np.random.randint(2):
                    x=x.flip(dims=(i,))
                    if i>=1:
                        y=y.flip(dims=(i-1,))
        return x,y#(uint8,uint8)


# Training

In [9]:
DataLoader_pth =      [  
                         '/kaggle/input/sparse-kidneys/DataLoader_kidney_2.pth', 
                         '/kaggle/input/sparse-kidneys/DataLoader_kidney_3_sparse.pth',
                         '/kaggle/input/sparse-kidneys/DataLoader_kidney_1_dense.pth',
                         '/kaggle/input/sparse-kidneys/DataLoader_kidney_3_dense.pth'
                      ]

tc.backends.cudnn.enabled = True
tc.backends.cudnn.benchmark = True

model=build_model()
model=DataParallel(model)

loss_fc=DiceLoss()
#loss_fn=nn.BCEWithLogitsLoss()
optimizer=tc.optim.AdamW(model.parameters(),lr=CFG.lr)
scaler=tc.cuda.amp.GradScaler()


            

for num, train_dataset_path in enumerate(DataLoader_pth):
    train_dataset = tc.load(train_dataset_path)
    scheduler = tc.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=CFG.lr,
                                                 steps_per_epoch=len(train_dataset), epochs=CFG.epochs+1,
                                                 pct_start=0.1)
    
    num_epochs_list = [5,10,25,25]
    epochs = num_epochs_list[num]
    for epoch in range(epochs):
        model.train()
        time = tqdm(total=len(train_dataset), desc=f"Training {train_dataset_path.split('/')[-1]}")
        losss, scores = 0, 0
        
        for i, (x, y) in enumerate(train_dataset):
            
            x=x.cuda().to(tc.float32)
            y=y.cuda().to(tc.float32)
            x=norm_with_clip(x.reshape(-1,*x.shape[2:])).reshape(x.shape)
            x=add_noise(x,max_randn_rate=0.5,x_already_normed=True)
        
            with autocast():
                pred=model(x)
                loss=loss_fc(pred,y)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            scheduler.step()
            score=dice_coef(pred.detach(),y)
            losss=(losss*i+loss.item())/(i+1)
            scores=(scores*i+score)/(i+1)
            time.set_description(f"epoch:{epoch},loss:{losss:.4f},score:{scores:.4f},lr{optimizer.param_groups[0]['lr']:.4e}")
            time.update()
            del loss,pred
    tc.save(model.module.state_dict(), f"model_after_{train_dataset_path.split('/')[-1]}_epoch_{epoch}.pt")
    del train_dataset
    time.close()

model_name Unet
backbone se_resnext50_32x4d


epoch:0,loss:0.9678,score:0.0467,lr2.1039e-05: 100%|██████████| 242/242 [01:59<00:00,  2.02it/s]

epoch:2,loss:0.5623,score:0.8116,lr5.9956e-05: 100%|██████████| 242/242 [01:42<00:00,  2.36it/s]

epoch:0,loss:0.1813,score:0.8408,lr2.1035e-05: 100%|██████████| 266/266 [02:00<00:00,  2.20it/s]

epoch:2,loss:0.1043,score:0.9005,lr5.9956e-05: 100%|██████████| 266/266 [01:53<00:00,  2.33it/s]

epoch:4,loss:0.0716,score:0.9233,lr5.8451e-05: 100%|██████████| 266/266 [01:54<00:00,  2.33it/s]

epoch:6,loss:0.0657,score:0.9209,lr5.4908e-05: 100%|██████████| 266/266 [01:53<00:00,  2.34it/s]

epoch:8,loss:0.0621,score:0.9234,lr4.9579e-05: 100%|██████████| 266/266 [01:54<00:00,  2.33it/s]

epoch:0,loss:0.0978,score:0.8362,lr2.1032e-05: 100%|██████████| 281/281 [02:08<00:00,  2.18it/s]

epoch:2,loss:0.0562,score:0.8771,lr5.9956e-05: 100%|██████████| 281/281 [02:00<00:00,  2.33it/s]

epoch:4,loss:0.0497,score:0.8909,lr5.8452e-05: 100%|██████████| 281/281 [02:00<00:00,  2.33it/s]

epoch:6,loss:0.0453,